In [6]:
include("ftdt.jl")
using .FTDT_LIB

#

########################################################################
# Se define la geometría en la ventana numérica principal
#######################################################################
und = 1*10^-6;                       # Unidad espacial
unt = 1*10^0;                       # Unidad temporal

epsz = (8.854187817*10^-12)*((und^3)/(unt^4)); 
muz = (4*pi*1e-7)*((unt^2)/und);
etaz=sqrt(muz/epsz);
cc =  sqrt(1/(epsz*muz));

freq=3.3e+14*(unt);                        # Frecuencia de la fuente de campo
lambda=cc/freq;                     # Longitud de onda de la fuente de campo 
omega=2.0*pi*freq;                  # Frecuencia en radiandes

dx = lambda/20;#(100e-3)*und;                             # Diferencial espacial

ventNum = round.([30*10^-3 15*10^-3]/und*dx,RoundUp)



ie= Int(ventNum[2]);                                     # nodos en y
je= Int(ventNum[1]);  


# Se agregan objetos en el medio
casoMaterial = "vacio";
if casoMaterial=="vacio"
  idx = 1:je;
  idy = 1:ie;
  medioProp = epsz;

end

simular(medioProp, idx, idy)

UndefVarError: UndefVarError: `simular` not defined